# Generating New Shoe Design using GAN
## Jerry Wei

**This notebook will:**
- Implement Generative Adversarial Networks to create new shoe designs
- Use matrix algebra concepts to achieve this goal

# 1 - Webscraping the Data

The data is first webscraped off of Zappos's collection of men's sneakers and atheltic shoes. Over 8000 images were collected, and then a final 2000 were selected to be used in the real data set. <br><br>
This cell runs through each page of Zappos's shoes and downloads each image it can find on that webpage and saves the images into a folder. This folder is then manually parsed through, and 2000 shoes were selected to be incorporated into the final data set. 

In [1]:
import urllib2
from os.path import basename
from urlparse import urlsplit
from bs4 import BeautifulSoup # for HTML parsing
global urlList
urlList = []

def downloadImages(url):
    """
    Downloads the image from Zappos's collection of men's shoes
    
    Arguments: 
    url -- The URL of the webpage to download images from
    
    Returns:
    N/A
    """
    
    print url
    global urlList
    if url in urlList: # prevent using the same URL again
        return
    urlList.append(url)
    try:
        urlContent = urllib2.urlopen(url).read()
    except:
        return

    soup = BeautifulSoup(''.join(urlContent), 'lxml')
    # find and download all images
    imgTags = soup.findAll('img')
    for imgTag in imgTags:
        imgUrl = imgTag['src']
        try:
            imgData = urllib2.urlopen(imgUrl).read()
            fileName = basename(urlsplit(imgUrl)[2])
            output = open(fileName,'wb')
            output.write(imgData)
            output.close()
        except:
            pass

for i in range (1, 80):
    downloadImages('https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=' + str(i), 1)

https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=1
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=2
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=3
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=4
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=5
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=6
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=7
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=8
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=9
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=10
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=11
https://www.zappos.com/men-sneakers-athletic-shoes/CK_XARC81wHAAQLiAgMBAhg.zso?p=12
h

# 2 - Resizing the Images

Because the images that were webscraped had different sizes, it is incredibly important to resize the images into a consistent shape, in this case 128 by 128 pixels. The resized images are then saved into another folder called resizedData. Additionally, the images are renamed in the function to allow easier identification for later purposes (each image is given a number from range 1 to 2000).

In [1]:
import os
import PIL
from PIL import Image

src = "./datav1" 
dst = "./resizedData" # resized

os.mkdir(dst)

counter = 1;
for each in os.listdir(src):
    if each == '.DS_Store':
        continue;
    img = Image.open('datav1/' + each)
    img = img.resize((128, 128), PIL.Image.ANTIALIAS)
    img.save('resizedData/' + str(counter) + '.jpg')
    counter+=1
    
print("Done")

Done


# 3 - Changing RGBA to RGB

The images are currently in RGBA form, and this cell will convert them from RGBA into RGB, the three layers that a convolutional neural network takes in. The new images are saved into another folder called resized_black. 

In [2]:
from PIL import Image
import os
src = "resizedData"
dst = "resized_black/"

os.mkdir(dst)

for each in os.listdir(src):
    png = Image.open(os.path.join(src,each))
    # print each
    if png.mode == 'RGBA':
        png.load() # required for png.split()
        background = Image.new("RGB", png.size, (0,0,0))
        background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
        background.save(os.path.join(dst,each.split('.')[0] + '.jpg'), 'JPEG')
    else:
        png.convert('RGB')
        png.save(os.path.join(dst,each.split('.')[0] + '.jpg'), 'JPEG')

print("Done")

Done


# 4 - Preparing for Model Training

This cell imports basic packages needed for the model's traning and also defines some variables that are needed for future uses.

In [1]:
import os
import tensorflow as tf
import numpy as np
import cv2
import random
import scipy.misc
from utils import *

slim = tf.contrib.slim

HEIGHT, WIDTH, CHANNEL = 128, 128, 3
BATCH_SIZE = 64
EPOCH = 100
os.environ['CUDA_VISIBLE_DEVICES'] = '15'
version = 'newShoes'
newShoe_path = './' + version

# 5 - Leaky ReLu Function

This cell simply defines a leaky ReLu function for later uses. <br> <br>
A leaky relu function allows a small, non-zero gradient when a unit is not active. This addresses the problem of "dying" relu units by allowing the units to maintain a small gradient when inactive instead of becoming 0.

In [2]:
def lrelu(x, n, leak=0.2): 
    """
    Leaky Relu function
    
    Arguments: 
    x -- The actual data to be applied into the leaky relu function
    n -- The name of the returned sequence
    leak -- The small amount of gradient applied when a unit is inactive
    
    Returns:
    result -- The end unit after being applied into the leaky relu function
    """
    
    result = tf.maximum(x, leak * x, name=n) 
    return result

# 6 - Processing the Data

In [3]:
def process_data():   
    current_dir = os.getcwd()
    # parent = os.path.dirname(current_dir)
    shoe_dir = os.path.join(current_dir, 'resized_black')
    images = []
    for each in os.listdir(shoe_dir):
        images.append(os.path.join(shoe_dir,each))
    # print images    
    all_images = tf.convert_to_tensor(images, dtype = tf.string)
    
    images_queue = tf.train.slice_input_producer(
                                        [all_images])
                                        
    content = tf.read_file(images_queue[0])
    image = tf.image.decode_jpeg(content, channels = CHANNEL)
    # sess1 = tf.Session()
    # print sess1.run(image)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta = 0.1)
    image = tf.image.random_contrast(image, lower = 0.9, upper = 1.1)
    # noise = tf.Variable(tf.truncated_normal(shape = [HEIGHT,WIDTH,CHANNEL], dtype = tf.float32, stddev = 1e-3, name = 'noise')) 
    # print image.get_shape()
    size = [HEIGHT, WIDTH]
    image = tf.image.resize_images(image, size)
    image.set_shape([HEIGHT,WIDTH,CHANNEL])
    # image = image + noise
    # image = tf.transpose(image, perm=[2, 0, 1])
    # print image.get_shape()
    
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    
    iamges_batch = tf.train.shuffle_batch(
                                    [image], batch_size = BATCH_SIZE,
                                    num_threads = 4, capacity = 200 + 3* BATCH_SIZE,
                                    min_after_dequeue = 200)
    num_images = len(images)

    return iamges_batch, num_images

# 7 - Creating the Generator Network

In [4]:
def generator(input, random_dim, is_train, reuse=False):
    c4, c8, c16, c32, c64 = 512, 256, 128, 64, 32 # channel num
    s4 = 4
    output_dim = CHANNEL  # RGB image
    with tf.variable_scope('gen') as scope:
        if reuse:
            scope.reuse_variables()
        w1 = tf.get_variable('w1', shape=[random_dim, s4 * s4 * c4], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1', shape=[c4 * s4 * s4], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        flat_conv1 = tf.add(tf.matmul(input, w1), b1, name='flat_conv1')
         #Convolution, bias, activation, repeat! 
        conv1 = tf.reshape(flat_conv1, shape=[-1, s4, s4, c4], name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn1')
        act1 = tf.nn.relu(bn1, name='act1')
        # 8*8*256
        #Convolution, bias, activation, repeat! 
        conv2 = tf.layers.conv2d_transpose(act1, c8, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = tf.nn.relu(bn2, name='act2')
        # 16*16*128
        conv3 = tf.layers.conv2d_transpose(act2, c16, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = tf.nn.relu(bn3, name='act3')
        # 32*32*64
        conv4 = tf.layers.conv2d_transpose(act3, c32, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = tf.nn.relu(bn4, name='act4')
        # 64*64*32
        conv5 = tf.layers.conv2d_transpose(act4, c64, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv5')
        bn5 = tf.contrib.layers.batch_norm(conv5, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn5')
        act5 = tf.nn.relu(bn5, name='act5')
        
        #128*128*3
        conv6 = tf.layers.conv2d_transpose(act5, output_dim, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv6')
        # bn6 = tf.contrib.layers.batch_norm(conv6, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn6')
        act6 = tf.nn.tanh(conv6, name='act6')
        return act6

# 8 -  Creating the Discriminator Network

In [5]:
def discriminator(input, is_train, reuse=False):
    c2, c4, c8, c16 = 64, 128, 256, 512  # channel num: 64, 128, 256, 512
    with tf.variable_scope('dis') as scope:
        if reuse:
            scope.reuse_variables()

        #Convolution, activation, bias, repeat! 
        conv1 = tf.layers.conv2d(input, c2, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training = is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope = 'bn1')
        act1 = lrelu(conv1, n='act1')
         #Convolution, activation, bias, repeat! 
        conv2 = tf.layers.conv2d(act1, c4, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = lrelu(bn2, n='act2')
        #Convolution, activation, bias, repeat! 
        conv3 = tf.layers.conv2d(act2, c8, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = lrelu(bn3, n='act3')
         #Convolution, activation, bias, repeat! 
        conv4 = tf.layers.conv2d(act3, c16, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = lrelu(bn4, n='act4')
       
        # start from act4
        dim = int(np.prod(act4.get_shape()[1:]))
        fc1 = tf.reshape(act4, shape=[-1, dim], name='fc1')
      
        
        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))

        # wgan just get rid of the sigmoid
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')
        # dcgan
        acted_out = tf.nn.sigmoid(logits)
        return logits #, acted_out

# 9 - Training the Model

In [6]:
def train():
    random_dim = 100
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
    with tf.variable_scope('input'):
        #real and fake image placholders
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    
    # wgan
    fake_image = generator(random_input, random_dim, is_train)
    
    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    
    d_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)  # This optimizes the discriminator.
    g_loss = -tf.reduce_mean(fake_result)  # This optimizes the generator.
            

    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if 'dis' in var.name]
    g_vars = [var for var in t_vars if 'gen' in var.name]
    # test
    # print(d_vars)
    trainer_d = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(d_loss, var_list=d_vars)
    trainer_g = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(g_loss, var_list=g_vars)
    # clip discriminator weights
    d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in d_vars]

    
    batch_size = BATCH_SIZE
    image_batch, samples_num = process_data()
    
    batch_num = int(samples_num / batch_size)
    total_batch = 0
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # continue training
    save_path = saver.save(sess, "/tmp/model.ckpt")
    ckpt = tf.train.latest_checkpoint('./model/' + version)
    saver.restore(sess, save_path)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    print('total training sample num:%d' % samples_num)
    print('batch size: %d, batch num per epoch: %d, epoch num: %d' % (batch_size, batch_num, EPOCH))
    print('start training...')
    for i in range(EPOCH):
        print("Epoch number: " + str(i))
        for j in range(batch_num):
            print(j)
            d_iters = 5
            g_iters = 1

            train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            for k in range(d_iters):
                print(k)
                train_image = sess.run(image_batch)
                #wgan clip weights
                sess.run(d_clip)
                
                # Update the discriminator
                _, dLoss = sess.run([trainer_d, d_loss],
                                    feed_dict={random_input: train_noise, real_image: train_image, is_train: True})

            # Update the generator
            for k in range(g_iters):
                # train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
                _, gLoss = sess.run([trainer_g, g_loss],
                                    feed_dict={random_input: train_noise, is_train: True})

            print 'train:[%d/%d],d_loss:%f,g_loss:%f' % (i, j, dLoss, gLoss)
            
        # save check point every 500 epoch
        if i%500 == 0:
            if not os.path.exists('./model/' + version):
                os.makedirs('./model/' + version)
            saver.save(sess, './model/' +version + '/' + str(i))  
        if i%1 == 0:
            # save images
            if not os.path.exists(newShoe_path):
                os.makedirs(newShoe_path)
            sample_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise, is_train: False})
            # imgtest = imgtest * 255.0
            # imgtest.astype(np.uint8)
            save_images(imgtest, [8,8] ,newShoe_path + '/epoch' + str(i) + '.jpg')
            
            print('train:[%d],d_loss:%f,g_loss:%f' % (i, dLoss, gLoss))
    coord.request_stop()
    coord.join(threads)

# 10 - Running the Model

In [8]:
import time
start_time = time.time()

train()

print("--- %s seconds ---" % (time.time() - start_time))

15
INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
total training sample num:1999
batch size: 64, batch num per epoch: 31, epoch num: 100
start training...
Epoch number: 0
0
0
1
2
3
4
train:[0/0],d_loss:-18.365097,g_loss:11.902464
1
0
1
2
3
4
train:[0/1],d_loss:-43.064754,g_loss:24.367065
2
0
1
2
3
4
train:[0/2],d_loss:-71.364304,g_loss:39.459084
3
0
1
2
3
4
train:[0/3],d_loss:-101.530640,g_loss:55.174210
4
0
1
2
3
4
train:[0/4],d_loss:-127.989517,g_loss:69.719460
5
0
1
2
3
4
train:[0/5],d_loss:-156.313446,g_loss:83.267746
6
0
1
2
3
4
train:[0/6],d_loss:-180.002487,g_loss:95.358788
7
0
1
2
3
4
train:[0/7],d_loss:-199.602905,g_loss:105.782867
8
0
1
2
3
4
train:[0/8],d_loss:-218.970444,g_loss:115.386917
9
0
1
2
3
4
train:[0/9],d_loss:-233.742966,g_loss:124.006180
10
0
1
2
3
4
train:[0/10],d_loss:-248.121338,g_loss:131.663696
11
0
1
2
3
4
train:[0/11],d_loss:-262.939545,g_loss:138.742081
12
0
1
2
3
4
train:[0/12],d_loss:-273.206116,g_loss:142.614319
13
0
1
2
3
4
train:[0/13],d_l

1
2
3
4
train:[4/1],d_loss:-375.077087,g_loss:189.449615
2
0
1
2
3
4
train:[4/2],d_loss:-373.225647,g_loss:188.678284
3
0
1
2
3
4
train:[4/3],d_loss:-374.101379,g_loss:189.001190
4
0
1
2
3
4
train:[4/4],d_loss:-373.862854,g_loss:188.698303
5
0
1
2
3
4
train:[4/5],d_loss:-368.216156,g_loss:187.944733
6
0
1
2
3
4
train:[4/6],d_loss:-370.060303,g_loss:185.253296
7
0
1
2
3
4
train:[4/7],d_loss:-356.339508,g_loss:185.454041
8
0
1
2
3
4
train:[4/8],d_loss:-375.373352,g_loss:189.855530
9
0
1
2
3
4
train:[4/9],d_loss:-374.874969,g_loss:189.295074
10
0
1
2
3
4
train:[4/10],d_loss:-365.147278,g_loss:182.154175
11
0
1
2
3
4
train:[4/11],d_loss:-344.666107,g_loss:183.777771
12
0
1
2
3
4
train:[4/12],d_loss:-206.067993,g_loss:79.059723
13
0
1
2
3
4
train:[4/13],d_loss:-77.980316,g_loss:-12.502007
14
0
1
2
3
4
train:[4/14],d_loss:-181.820786,g_loss:5.910035
15
0
1
2
3
4
train:[4/15],d_loss:-46.012314,g_loss:94.019958
16
0
1
2
3
4
train:[4/16],d_loss:-182.417770,g_loss:-158.719330
17
0
1
2
3
4
train:

1
2
3
4
train:[8/5],d_loss:-360.615723,g_loss:181.649796
6
0
1
2
3
4
train:[8/6],d_loss:-345.707092,g_loss:187.539215
7
0
1
2
3
4
train:[8/7],d_loss:-352.238403,g_loss:178.356949
8
0
1
2
3
4
train:[8/8],d_loss:-321.721252,g_loss:184.961914
9
0
1
2
3
4
train:[8/9],d_loss:-367.684174,g_loss:188.268143
10
0
1
2
3
4
train:[8/10],d_loss:-355.372070,g_loss:184.861023
11
0
1
2
3
4
train:[8/11],d_loss:-342.649811,g_loss:175.714508
12
0
1
2
3
4
train:[8/12],d_loss:-181.163330,g_loss:170.770996
13
0
1
2
3
4
train:[8/13],d_loss:-361.708221,g_loss:187.463623
14
0
1
2
3
4
train:[8/14],d_loss:-361.135956,g_loss:188.383774
15
0
1
2
3
4
train:[8/15],d_loss:-363.736938,g_loss:186.465149
16
0
1
2
3
4
train:[8/16],d_loss:-312.592987,g_loss:175.527893
17
0
1
2
3
4
train:[8/17],d_loss:-357.268494,g_loss:188.066498
18
0
1
2
3
4
train:[8/18],d_loss:-366.545135,g_loss:189.338623
19
0
1
2
3
4
train:[8/19],d_loss:-368.281494,g_loss:188.094925
20
0
1
2
3
4
train:[8/20],d_loss:-314.278748,g_loss:177.314468
21
0
1

1
2
3
4
train:[12/8],d_loss:-272.366455,g_loss:148.399628
9
0
1
2
3
4
train:[12/9],d_loss:-215.898666,g_loss:167.276428
10
0
1
2
3
4
train:[12/10],d_loss:-359.152069,g_loss:186.503876
11
0
1
2
3
4
train:[12/11],d_loss:-222.244797,g_loss:171.347046
12
0
1
2
3
4
train:[12/12],d_loss:-362.090637,g_loss:188.194244
13
0
1
2
3
4
train:[12/13],d_loss:-341.679291,g_loss:174.515137
14
0
1
2
3
4
train:[12/14],d_loss:-259.187500,g_loss:-53.287231
15
0
1
2
3
4
train:[12/15],d_loss:-273.555115,g_loss:183.439880
16
0
1
2
3
4
train:[12/16],d_loss:-366.153870,g_loss:189.345154
17
0
1
2
3
4
train:[12/17],d_loss:-371.230591,g_loss:189.193115
18
0
1
2
3
4
train:[12/18],d_loss:-371.481659,g_loss:190.429230
19
0
1
2
3
4
train:[12/19],d_loss:-366.294952,g_loss:185.093964
20
0
1
2
3
4
train:[12/20],d_loss:-281.466309,g_loss:147.573578
21
0
1
2
3
4
train:[12/21],d_loss:-148.639160,g_loss:166.907837
22
0
1
2
3
4
train:[12/22],d_loss:-239.500748,g_loss:65.586594
23
0
1
2
3
4
train:[12/23],d_loss:-280.346375,g_l

1
2
3
4
train:[16/10],d_loss:-360.305969,g_loss:184.043381
11
0
1
2
3
4
train:[16/11],d_loss:-363.737030,g_loss:183.423111
12
0
1
2
3
4
train:[16/12],d_loss:-286.564392,g_loss:178.257690
13
0
1
2
3
4
train:[16/13],d_loss:-299.244293,g_loss:148.261414
14
0
1
2
3
4
train:[16/14],d_loss:-365.714478,g_loss:188.501587
15
0
1
2
3
4
train:[16/15],d_loss:-371.428101,g_loss:190.000061
16
0
1
2
3
4
train:[16/16],d_loss:-372.624023,g_loss:189.783279
17
0
1
2
3
4
train:[16/17],d_loss:-373.264526,g_loss:189.405090
18
0
1
2
3
4
train:[16/18],d_loss:-371.258850,g_loss:187.370483
19
0
1
2
3
4
train:[16/19],d_loss:-372.342041,g_loss:189.528122
20
0
1
2
3
4
train:[16/20],d_loss:-364.046448,g_loss:186.764801
21
0
1
2
3
4
train:[16/21],d_loss:-363.069031,g_loss:180.348801
22
0
1
2
3
4
train:[16/22],d_loss:-195.073929,g_loss:182.001709
23
0
1
2
3
4
train:[16/23],d_loss:-350.776733,g_loss:184.806122
24
0
1
2
3
4
train:[16/24],d_loss:-366.143433,g_loss:186.820496
25
0
1
2
3
4
train:[16/25],d_loss:-367.520966

...

1
2
3
4
train:[81/10],d_loss:-255.186203,g_loss:121.460930
11
0
1
2
3
4
train:[81/11],d_loss:-283.286316,g_loss:175.635620
12
0
1
2
3
4
train:[81/12],d_loss:-189.982727,g_loss:122.724701
13
0
1
2
3
4
train:[81/13],d_loss:-250.614380,g_loss:174.453262
14
0
1
2
3
4
train:[81/14],d_loss:-312.078003,g_loss:158.137772
15
0
1
2
3
4
train:[81/15],d_loss:-276.990082,g_loss:170.734283
16
0
1
2
3
4
train:[81/16],d_loss:-302.282532,g_loss:146.797180
17
0
1
2
3
4
train:[81/17],d_loss:-232.769836,g_loss:173.729980
18
0
1
2
3
4
train:[81/18],d_loss:-254.012573,g_loss:116.809402
19
0
1
2
3
4
train:[81/19],d_loss:-310.460846,g_loss:175.496841
20
0
1
2
3
4
train:[81/20],d_loss:-311.036591,g_loss:159.851944
21
0
1
2
3
4
train:[81/21],d_loss:-284.047852,g_loss:175.352524
22
0
1
2
3
4
train:[81/22],d_loss:-288.510498,g_loss:143.984116
23
0
1
2
3
4
train:[81/23],d_loss:-263.800598,g_loss:172.105957
24
0
1
2
3
4
train:[81/24],d_loss:-312.782043,g_loss:145.835754
25
0
1
2
3
4
train:[81/25],d_loss:-294.215332

1
2
3
4
train:[85/12],d_loss:-309.824829,g_loss:164.451813
13
0
1
2
3
4
train:[85/13],d_loss:-171.671082,g_loss:159.646744
14
0
1
2
3
4
train:[85/14],d_loss:-255.841812,g_loss:107.440338
15
0
1
2
3
4
train:[85/15],d_loss:-241.505066,g_loss:166.524063
16
0
1
2
3
4
train:[85/16],d_loss:-303.872192,g_loss:156.750763
17
0
1
2
3
4
train:[85/17],d_loss:-275.517273,g_loss:172.201416
18
0
1
2
3
4
train:[85/18],d_loss:-295.284698,g_loss:164.617249
19
0
1
2
3
4
train:[85/19],d_loss:-310.613220,g_loss:156.122406
20
0
1
2
3
4
train:[85/20],d_loss:-174.427170,g_loss:150.491592
21
0
1
2
3
4
train:[85/21],d_loss:-265.655212,g_loss:99.883148
22
0
1
2
3
4
train:[85/22],d_loss:-245.211517,g_loss:165.821732
23
0
1
2
3
4
train:[85/23],d_loss:-307.567841,g_loss:159.390442
24
0
1
2
3
4
train:[85/24],d_loss:-220.071228,g_loss:162.770538
25
0
1
2
3
4
train:[85/25],d_loss:-297.158630,g_loss:152.791870
26
0
1
2
3
4
train:[85/26],d_loss:-209.353714,g_loss:162.939926
27
0
1
2
3
4
train:[85/27],d_loss:-257.656921,

1
2
3
4
train:[89/14],d_loss:-290.057495,g_loss:146.177429
15
0
1
2
3
4
train:[89/15],d_loss:-249.925537,g_loss:161.756271
16
0
1
2
3
4
train:[89/16],d_loss:-265.770569,g_loss:132.310349
17
0
1
2
3
4
train:[89/17],d_loss:-228.531952,g_loss:162.167603
18
0
1
2
3
4
train:[89/18],d_loss:-258.254150,g_loss:90.148407
19
0
1
2
3
4
train:[89/19],d_loss:-253.823212,g_loss:163.867065
20
0
1
2
3
4
train:[89/20],d_loss:-290.188873,g_loss:141.642120
21
0
1
2
3
4
train:[89/21],d_loss:-265.329956,g_loss:171.287994
22
0
1
2
3
4
train:[89/22],d_loss:-302.175537,g_loss:145.898041
23
0
1
2
3
4
train:[89/23],d_loss:-254.589569,g_loss:156.710251
24
0
1
2
3
4
train:[89/24],d_loss:-285.376221,g_loss:153.065964
25
0
1
2
3
4
train:[89/25],d_loss:-243.335251,g_loss:169.561035
26
0
1
2
3
4
train:[89/26],d_loss:-289.832794,g_loss:124.754105
27
0
1
2
3
4
train:[89/27],d_loss:-244.901596,g_loss:154.291748
28
0
1
2
3
4
train:[89/28],d_loss:-285.570740,g_loss:164.820862
29
0
1
2
3
4
train:[89/29],d_loss:-223.247040,

1
2
3
4
train:[93/16],d_loss:-288.498779,g_loss:156.262024
17
0
1
2
3
4
train:[93/17],d_loss:-235.799652,g_loss:159.209213
18
0
1
2
3
4
train:[93/18],d_loss:-251.654770,g_loss:125.212372
19
0
1
2
3
4
train:[93/19],d_loss:-228.470184,g_loss:155.665283
20
0
1
2
3
4
train:[93/20],d_loss:-266.501740,g_loss:104.466454
21
0
1
2
3
4
train:[93/21],d_loss:-267.981903,g_loss:166.956985
22
0
1
2
3
4
train:[93/22],d_loss:-295.705566,g_loss:132.657211
23
0
1
2
3
4
train:[93/23],d_loss:-235.929321,g_loss:165.879684
24
0
1
2
3
4
train:[93/24],d_loss:-311.820251,g_loss:169.920288
25
0
1
2
3
4
train:[93/25],d_loss:-241.628082,g_loss:172.160400
26
0
1
2
3
4
train:[93/26],d_loss:-255.317780,g_loss:110.286530
27
0
1
2
3
4
train:[93/27],d_loss:-300.752258,g_loss:171.970764
28
0
1
2
3
4
train:[93/28],d_loss:-311.394958,g_loss:163.895142
29
0
1
2
3
4
train:[93/29],d_loss:-257.988342,g_loss:172.371399
30
0
1
2
3
4
train:[93/30],d_loss:-163.196548,g_loss:56.127472
train:[93],d_loss:-163.196548,g_loss:56.127472

1
2
3
4
train:[97/18],d_loss:-235.769043,g_loss:80.986206
19
0
1
2
3
4
train:[97/19],d_loss:-242.249130,g_loss:155.732941
20
0
1
2
3
4
train:[97/20],d_loss:-269.386597,g_loss:96.302200
21
0
1
2
3
4
train:[97/21],d_loss:-300.056610,g_loss:175.765778
22
0
1
2
3
4
train:[97/22],d_loss:-329.585144,g_loss:167.222473
23
0
1
2
3
4
train:[97/23],d_loss:-250.038269,g_loss:171.070404
24
0
1
2
3
4
train:[97/24],d_loss:-236.770584,g_loss:71.182472
25
0
1
2
3
4
train:[97/25],d_loss:-254.342499,g_loss:173.276306
26
0
1
2
3
4
train:[97/26],d_loss:-198.770569,g_loss:67.264435
27
0
1
2
3
4
train:[97/27],d_loss:-283.655029,g_loss:175.581635
28
0
1
2
3
4
train:[97/28],d_loss:-237.133499,g_loss:62.116165
29
0
1
2
3
4
train:[97/29],d_loss:-266.776550,g_loss:169.303314
30
0
1
2
3
4
train:[97/30],d_loss:-214.885040,g_loss:111.724747
train:[97],d_loss:-214.885040,g_loss:111.724747
Epoch number: 98
0
0
1
2
3
4
train:[98/0],d_loss:-237.946838,g_loss:173.077087
1
0
1
2
3
4
train:[98/1],d_loss:-271.766632,g_loss: